In [168]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime

In [169]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [170]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select Id, Name, IdentificationId__pc, Identification_Type__c,Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,poliza__c,Empleador__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    ["Id", "Name", "IdentificationId__pc", "Identification_Type__c", "Holding__c", "Pa_s_del_Paciente__c",'Campa_a__c','AccountNumber','Id_Empresa__c','Empleador__c','poliza__c']
]
Empresas_SF.shape

(7877, 11)

In [171]:
# recuperar todos los pacientes ID
results = sf.query_all(
    """
  select Id, IdentificationId__pc,Name,poliza__c, Identification_Type__c, IsPersonAccount, Ocupaci_n__c, Pa_s_del_Paciente__c, PersonTitle ,PersonDepartment  from Account where ispersonaccount = True
    """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[["Id", "IdentificationId__pc", "Name",'poliza__c','Identification_Type__c','IsPersonAccount','Ocupaci_n__c','Pa_s_del_Paciente__c','PersonTitle','PersonDepartment']]
Pacientes_SF.shape

In [ ]:
# recuperar todos los pacientes ID
results = sf.query_all(
    """
   select Id,Name,AccountId  from Contact
    """
)
Pacientesc_SF = pd.DataFrame(results["records"])

Pacientesc_SF = Pacientesc_SF[["Id", "AccountId", "Name"]]
Pacientesc_SF.shape

In [ ]:
Pacientesc_SF.head()

In [ ]:
#Relacion Account Contact Relacion
results = sf.query_all(
    """
   select Id,AccountId,ContactId,NameAccount__c, RutContact__c from AccountContactRelation
    """
)
Relacion_Pacientes_Empresas = pd.DataFrame(results["records"])

Relacion_Pacientes_Empresas = Relacion_Pacientes_Empresas[
    ["Id", "AccountId", "ContactId", "NameAccount__c", "RutContact__c"]
]
Relacion_Pacientes_Empresas.shape

In [ ]:
#union de pacientes
Union_account_contact_relation = pd.merge(
    left=Relacion_Pacientes_Empresas,
    right=Pacientesc_SF,
    how="left",
    left_on="ContactId",
    right_on="Id",
)
Union_account_contact_relation.head()

In [ ]:
#union de empresas
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Empresas_SF,
    how="left",
    left_on="AccountId_x",
    right_on="Id",
)
Union_account_contact_relation.head()

In [ ]:
#union de pacientes con sus datos
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Pacientes_SF,
    how="left",
    left_on="RutContact__c",
    right_on="IdentificationId__pc",
)
Union_account_contact_relation.head()

In [ ]:
nuevos = "F:/Empresas SF/carga/sacarid.xlsx"
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

    

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")
    # aca se trabaja con el nuevo rut desde rut1 ojooooooo    
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()    
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()


In [ ]:
pacientes_y_datos = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
pacientes_y_datos.to_excel("F:/Empresas SF/carga/pacientes_en_account.xlsx", index=False)

In [ ]:
pacientes_y_datos.head()

In [ ]:
pacientes_y_datos2 = pd.merge(
    left=pacientes_y_datos,
    right=Pacientesc_SF,
    how="left",
    left_on="Id",
    right_on="AccountId",
)
pacientes_y_datos2.to_excel("F:/Empresas SF/carga/pacientes_en_Contact.xlsx", index=False)

In [ ]:

    


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"]
)


Final_pacientes_y_empresa = Pacientes_Archivo

Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Final_pacientes_y_empresa = Final_pacientes_y_empresa[
    Final_pacientes_y_empresa["RutEstaoNo"] == False
]

if ("nombres" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["FirstName"] = Final_pacientes_y_empresa["nombres"]
if ("apellidos" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["LastName"] = Final_pacientes_y_empresa["apellidos"]
if ("rut" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Final_pacientes_y_empresa["rut"]
if ("email" in Final_pacientes_y_empresa.columns) == True:
    Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].str.lower()
    Subir_pacientes["PersonEmail"] = Final_pacientes_y_empresa["email"]
if ("nacimiento" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Final_pacientes_y_empresa["nacimiento"]
if ("sexo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Final_pacientes_y_empresa["sexo"]
if ("direccion" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
if ("cargo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonTitle"] = Final_pacientes_y_empresa["cargo"]
if ("division" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonDepartment"] = Final_pacientes_y_empresa["division"]

Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


Subir_pacientes.to_excel("F:/Empresas SF/carga/pacientes_q_no_estan.xlsx", index=False)



In [ ]:

Union_account_contact_relation['IdentificationId__pc_y'] = Union_account_contact_relation['IdentificationId__pc_y'].astype(str)
Union_account_contact_relation['IdentificationId__pc_y'] = Union_account_contact_relation['IdentificationId__pc_y'].str.replace("-", "")
Union_account_contact_relation['IdentificationId__pc_y'] = Union_account_contact_relation['IdentificationId__pc_y'].str.upper()
Union_account_contact_relation['IdentificationId__pc_y'] = Union_account_contact_relation['IdentificationId__pc_y'].str.strip()


pacientes_y_datos_relation = pd.merge(
    left=Pacientes_Archivo,
    right=Union_account_contact_relation,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc_y",
)
pacientes_y_datos_relation.head()
pacientes_y_datos_relation.to_excel("F:/Empresas SF/carga/pacientes_en_Acountcontacrelation.xlsx", index=False)

In [ ]:
#buscar empresas desde Programas_por_Empresas__c
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

In [ ]:
#unir empresas y todas sus caracteristicas


empresas_unidas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
empresas_unidas.shape

In [ ]:
empresas_unidas.to_excel("F:/Empresas SF/carga/Empresas.xlsx", index=False)

In [ ]:
empresas_unidas.head()

In [ ]:
#buscar empresas desde Paciente_Programas__c
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

In [ ]:
Todos_los_Pacientes_con_empresas = pd.merge(
    left=Paciente_Programas_SF,
    right=empresas_unidas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id_y",
)
Todos_los_Pacientes_con_empresas.shape

In [ ]:
Todos_los_Pacientes_con_empresas.head()

In [ ]:
Todos_los_Pacientes_con_empresas2 = pd.merge(
    left=Pacientes_SF,
    right=Todos_los_Pacientes_con_empresas,
    how="left",
    left_on="Id",
    right_on="Paciente__c",
)
Todos_los_Pacientes_con_empresas2.shape

In [ ]:
Todos_los_Pacientes_con_empresas2.head()

In [ ]:
#Empresa_Padre__c


Todos_los_Pacientes_con_empresas4 = Todos_los_Pacientes_con_empresas2[Todos_los_Pacientes_con_empresas2['Campa_a__c']== 'Forus']
Todos_los_Pacientes_con_empresas4.shape


In [ ]:
Todos_los_Pacientes_con_empresas4.to_excel("F:/Empresas SF/carga/Forus.xlsx", index=False)